---
# 6. SISTEMA DE RECOMENDACIÓN USANDO MODELOS PRE-ENTRENADOS (TRANSFORMERS) DE HUGGING FACE


In [19]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

<div style="text-align: justify;">

A continuación, vamos a desarrollar un sistema de recomendación que aprovecha el contenido textual enriquecido de nuestro dataset. Para ello, primero cargamos y preparamos el DataFrame a partir del archivo CSV 'datasets/clean_description_english.csv'. Hemos renombrado el DataFrame a `df_hf`. En este paso, combinamos información relevante —como el título, la descripción limpia, la puntuación media, las categorías y el autor— para formar una nueva columna "description". La idea es que esta columna consolidada nos brinde un contexto completo del libro, lo que resulta muy útil a la hora de comparar la consulta del usuario con las características de cada libro.

La estrategia que seguimos consiste en utilizar modelos de SentenceTransformer para generar embeddings de cada descripción. Los embeddings son vectores numéricos que capturan el significado semántico de los textos. En nuestro caso, empleamos el modelo 'all-MiniLM-L6-v2' por su eficiencia y capacidad para generar representaciones precisas de oraciones breves. Con este modelo, convertimos la columna "description" en una lista de strings y generamos un conjunto de embeddings, llamados `book_embeddings`, que utilizaremos para medir la similitud entre la consulta del usuario y las descripciones de los libros.

Definimos la función `get_recommendations` que, dada una consulta y los embeddings de los libros, realiza los siguientes pasos:
- Genera el embedding de la consulta utilizando el mismo modelo.
- Calcula la similitud coseno entre el embedding de la consulta y todos los embeddings del dataset.
- Ordena los libros en función de la similitud obtenida y devuelve los libros con mayor coincidencia.

Este método permite que, al ingresar una consulta como "A book to explore meditation and mindfulness practices.", el sistema nos devuelva recomendaciones basadas en la cercanía semántica entre la consulta y las descripciones enriquecidas.

Posteriormente, se experimenta con otros modelos de SentenceTransformer, como 'bert-large-nli-mean-tokens' y 'all-roberta-large-v1', para explorar diferentes formas de generar embeddings y ver cómo varían las recomendaciones con distintas arquitecturas. Cada modelo ofrece una perspectiva distinta sobre el significado de los textos, lo que puede ser muy útil para adaptar el recomendador a diversas necesidades o contextos.

En resumen, este enfoque se basa en la idea de enriquecer la descripción de cada libro combinando varias fuentes de información (título, resumen, puntuación, categorías y autor) y utilizar modelos de SentenceTransformer para obtener representaciones vectoriales que capturen su significado. Con estas representaciones, podemos comparar la consulta del usuario con el contenido de cada libro y sugerir aquellos que son semánticamente más afines

</div>

In [5]:
df_hf = pd.read_csv('datasets/clean_description_english.csv')
df_hf

,author,desc,genre,rating,reviews,title,totalratings,idioma,clean_description
0,Helen Fremont,"""To this day, I don't even know what my mother...","Autobiography,Memoir,Nonfiction,World War II,H...",4.00,328,After Long Silence,2898,en,"To this day, I don't even know what my mother'..."
1,Christopher Morley,"Roger Mifflin is part pixie, part sage, part n...","Fiction,Writing,Books About Books,Classics,Hum...",4.03,1214,Parnassus on Wheels,6107,en,"Roger Mifflin is part pixie, part sage, part n..."
2,Robert J. Shiller,As Robert Shillerâ€™s new 2009 preface to his ...,"Economics,Economics,Finance,Business,Nonfictio...",3.98,275,Irrational Exuberance,7019,en,As Robert Shillers new 2009 preface to his pre...
3,Joseph Staten,"This is how it began...,It is the year 2524. H...","Science Fiction,Fiction,Games,Video Games,Spor...",3.98,271,Halo: Contact Harvest,8033,en,"This is how it began...,It is the year 2524. H..."
4,Richard Wiseman,"Is luck just fate, ,or can you change it? ,A g...","Psychology,Nonfiction,Self Help,Science,Self H...",3.82,146,"The Luck Factor: Changing Your Luck, Changing ...",1301,en,"Is luck just fate, ,or can you change it ,A gr..."
...,...,...,...,...,...,...,...,...,...
17447,Robert Jay Lifton,Nazi doctors did more than conduct bizarre exp...,"History,Nonfiction,Psychology,World War II,Hol...",4.10,118,The Nazi Doctors: Medical Killing and the Psyc...,3336,en,Nazi doctors did more than conduct bizarre exp...
17448,Priscilla Glenn,Ever since the one time Andie Weber threw caut...,"New Adult,Romance,Romance,Contemporary Romance...",3.90,450,Emancipating Andie,3974,en,Ever since the one time Andie Weber threw caut...
17449,Amber Kell,"Book one in The Thresl Chronicles Series,When ...","Romance,M M Romance,Science Fiction,Paranormal...",3.90,140,Soldier Mine,1770,en,"Book one in The Thresl Chronicles Series,When ..."
17450,Melissa Lucashenko,A darkly funny novel of romantic love and cult...,"Fiction,Cultural,Australia,Literary Fiction,Co...",3.93,164,Mullumbimby,1257,en,A darkly funny novel of romantic love and cult...


In [6]:
# Dado que vamos a hacer algunos cambios en la columna "descriptions", vamos a renombrar el dataframe a df_hf
df_hf = dataset.copy()

df_hf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17452 entries, 0 to 17451
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   author             17452 non-null  object 
 1   desc               17452 non-null  object 
 2   genre              17452 non-null  object 
 3   rating             17452 non-null  float64
 4   reviews            17452 non-null  int64  
 5   title              17452 non-null  object 
 6   totalratings       17452 non-null  int64  
 7   idioma             17452 non-null  object 
 8   clean_description  17452 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 1.2+ MB


La columna "description" se crea para consolidar en un solo campo toda la información relevante de cada libro. Al concatenar el título, la descripción limpia, la puntuación media, las categorías y el autor, se obtiene un contexto completo y enriquecido para cada registro. Esto es crucial para la generación de embeddings, ya que permite al modelo capturar no solo el contenido textual del resumen, sino también otros aspectos importantes que definen el libro, como su autor y clasificación temática. En esencia, esta estrategia mejora la capacidad del sistema de recomendación para identificar similitudes semánticas entre libros, ya que dispone de una representación más rica y detallada del contenido de cada documento.

In [7]:

df_hf["description"] = 'Title: ' + df_hf["title"] +'. ' + 'Desc / Summary: ' + df_hf["clean_description"] + '. ' + 'Average Rating: ' + df_hf["rating"].astype('str') + '. ' + 'Categories: ' + df_hf["genre"] + '. ' + 'Author: ' + df_hf["author"] + '.'

df_hf

,author,desc,genre,rating,reviews,title,totalratings,idioma,clean_description,description
0,Helen Fremont,"""To this day, I don't even know what my mother...","Autobiography,Memoir,Nonfiction,World War II,H...",4.00,328,After Long Silence,2898,en,"To this day, I don't even know what my mother'...",Title: After Long Silence. Desc / Summary: To ...
1,Christopher Morley,"Roger Mifflin is part pixie, part sage, part n...","Fiction,Writing,Books About Books,Classics,Hum...",4.03,1214,Parnassus on Wheels,6107,en,"Roger Mifflin is part pixie, part sage, part n...",Title: Parnassus on Wheels. Desc / Summary: Ro...
2,Robert J. Shiller,As Robert Shillerâ€™s new 2009 preface to his ...,"Economics,Economics,Finance,Business,Nonfictio...",3.98,275,Irrational Exuberance,7019,en,As Robert Shillers new 2009 preface to his pre...,Title: Irrational Exuberance. Desc / Summary: ...
3,Joseph Staten,"This is how it began...,It is the year 2524. H...","Science Fiction,Fiction,Games,Video Games,Spor...",3.98,271,Halo: Contact Harvest,8033,en,"This is how it began...,It is the year 2524. H...",Title: Halo: Contact Harvest. Desc / Summary: ...
4,Richard Wiseman,"Is luck just fate, ,or can you change it? ,A g...","Psychology,Nonfiction,Self Help,Science,Self H...",3.82,146,"The Luck Factor: Changing Your Luck, Changing ...",1301,en,"Is luck just fate, ,or can you change it ,A gr...","Title: The Luck Factor: Changing Your Luck, Ch..."
...,...,...,...,...,...,...,...,...,...,...
17447,Robert Jay Lifton,Nazi doctors did more than conduct bizarre exp...,"History,Nonfiction,Psychology,World War II,Hol...",4.10,118,The Nazi Doctors: Medical Killing and the Psyc...,3336,en,Nazi doctors did more than conduct bizarre exp...,Title: The Nazi Doctors: Medical Killing and t...
17448,Priscilla Glenn,Ever since the one time Andie Weber threw caut...,"New Adult,Romance,Romance,Contemporary Romance...",3.90,450,Emancipating Andie,3974,en,Ever since the one time Andie Weber threw caut...,Title: Emancipating Andie. Desc / Summary: Eve...
17449,Amber Kell,"Book one in The Thresl Chronicles Series,When ...","Romance,M M Romance,Science Fiction,Paranormal...",3.90,140,Soldier Mine,1770,en,"Book one in The Thresl Chronicles Series,When ...",Title: Soldier Mine. Desc / Summary: Book one ...
17450,Melissa Lucashenko,A darkly funny novel of romantic love and cult...,"Fiction,Cultural,Australia,Literary Fiction,Co...",3.93,164,Mullumbimby,1257,en,A darkly funny novel of romantic love and cult...,Title: Mullumbimby. Desc / Summary: A darkly f...


**Idea**: *Para construir el sistema de recomendación, compararemos la consulta del usuario con la nueva columna "description".*

---
## 6.1. Modelo pre-entrenado **all-MiniLM-L6-v2**

Elegimos el modelo 'all-MiniLM-L6-v2' porque combina una gran eficiencia con alta calidad, generando embeddings de frases que capturan de forma precisa el significado de textos breves. Se puede encontrar más información sobre el modelo usado en: https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2 


### Uso de SentenceTransformer para Generar Embeddings


Vamos a importar la clase `SentenceTransformer` desde la librería `sentence-transformers`.  
Esta clase integra internamente:
- El tokenizador.
- El modelo preentrenado.
- Un mecanismo de pooling que convierte las salidas del modelo en un único vector para cada oración.

#### Generación de Embeddings

- **`df['description'].tolist()`**  
  Convierte la columna `description` del dataframe en una lista de *strings*.

- **`model.encode(...)`**  
  Este método procesa cada descripción realizando los siguientes pasos:
  - Aplica internamente la tokenización.
  - Pasa los tokens por el modelo preentrenado.
  - Realiza un pooling para obtener un vector de características único.

- **Resultado: `book_embeddings`**  
  Se obtiene una colección de *embeddings* que representan el significado semántico de cada descripción.  
  Estos vectores puedlos vamos a  utilizar para comparar la similitud de la query del usuario con la descripcion del libro.

In [ ]:
!pip install tf-keras

In [10]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

book_embeddings = model.encode(df_hf['description'].tolist())

In [11]:
def get_recommendations(query, embeddings, df, model, top_n=5):
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, embeddings)
    top_indices = similarities[0].argsort()[-top_n:][::-1]
    return df.iloc[top_indices]

In [15]:
query = "A book to explore meditation and mindfulness practices."
recommendations = get_recommendations(query, book_embeddings, df_hf, model, 5)
recommendations

,author,desc,genre,rating,reviews,title,totalratings,idioma,clean_description,description
3428,"J. Mark G. Williams,Danny Penman,Jon Kabat-Zinn","From one of the leading thinkers on Mindfulness-Based Cognitive Therapy, a pioneering set of simple practices to dissolve anxiety, stress, exhaustion, and unhappiness.In ""Mindfulness,"" Oxford professor Mark Williams and award-winning journalist Dr. Danny Penman reveal the secrets to living a happier and less anxious, stressful and exhausting life. Based on the techniques of Mindfulness-Based Cognitive Therapy, the unique program developed by Williams and his colleagues, the book offers simple and straightforward forms of mindfulness meditation that can be done by anyone--and it can take just 10-20 minutes a day for the full benefits to be revealed.","Nonfiction,Psychology,Self Help,Self Help,Personal Development,Health,Spirituality,Philosophy,Health,Mental Health,Religion,Buddhism,Unfinished",3.90,545,Mindfulness: An Eight-Week Plan for Finding Peace in a Frantic World,13471,en,"From one of the leading thinkers on MindfulnessBased Cognitive Therapy, a pioneering set of simple practices to dissolve anxiety, stress, exhaustion, and unhappiness.In Mindfulness, Oxford professor Mark Williams and awardwinning journalist Dr. Danny Penman reveal the secrets to living a happier and less anxious, stressful and exhausting life. Based on the techniques of MindfulnessBased Cognitive Therapy, the unique program developed by Williams and his colleagues, the book offers simple and straightforward forms of mindfulness meditation that can be done by anyoneand it can take just 1020 minutes a day for the full benefits to be revealed.","Title: Mindfulness: An Eight-Week Plan for Finding Peace in a Frantic World. Desc / Summary: From one of the leading thinkers on MindfulnessBased Cognitive Therapy, a pioneering set of simple practices to dissolve anxiety, stress, exhaustion, and unhappiness.In Mindfulness, Oxford professor Mark Williams and awardwinning journalist Dr. Danny Penman reveal the secrets to living a happier and less anxious, stressful and exhausting life. Based on the techniques of MindfulnessBased Cognitive Therapy, the unique program developed by Williams and his colleagues, the book offers simple and straightforward forms of mindfulness meditation that can be done by anyoneand it can take just 1020 minutes a day for the full benefits to be revealed.. Average Rating: 3.9. Categories: Nonfiction,Psychology,Self Help,Self Help,Personal Development,Health,Spirituality,Philosophy,Health,Mental Health,Religion,Buddhism,Unfinished. Author: J. Mark G. Williams,Danny Penman,Jon Kabat-Zinn."
7500,Thich Nhat Hanh,"Find peace and calm amid the busyness of your life with this new book by Zen Master Thich Nhat Hanh. Designed to be both inspiration and guidebook for those new to mindfulness practice, ,Making Space, offers easy-to-follow instructions for setting up a breathing room, listening to a bell, sitting, breathing, and walking meditations, and cooking and eating a meal in mindfulness. Whether you live alone or with a family, this beautifully illustrated book can help you create a sense of retreat and sanctuary at home.","Nonfiction,Religion,Buddhism,Spirituality,Self Help,Philosophy,Self Help,Personal Development,Buddhism,Zen,Audiobook,Religion,Health",4.08,106,Making Space: Creating a Home Meditation Practice,1291,en,"Find peace and calm amid the busyness of your life with this new book by Zen Master Thich Nhat Hanh. Designed to be both inspiration and guidebook for those new to mindfulness practice, ,Making Space, offers easytofollow instructions for setting up a breathing room, listening to a bell, sitting, breathing, and walking meditations, and cooking and eating a meal in mindfulness. Whether you live alone or with a family, this beautifully illustrated book can help you create a sense of retreat and sanctuary at home.","Title: Making Space: Creating a Home Meditation Practice. Desc / Sum

Para observar las respuestas, vamos a observar la columna 'description' del dataframe completa: 

In [13]:
pd.set_option('display.max_colwidth', None)

# Ahora imprime las recomendaciones y verás la columna 'description' completa.
recommendations

,author,desc,genre,rating,reviews,title,totalratings,idioma,clean_description,description
14528,Richard Brautigan,"A reclusive young man works in a San Francisco library for unpublishable books. Life's losers, an astonishing number of whom seem to be writers, can bring their manuscripts to the library, where they will be welcomed, registered and shelved. They will not be read, but they will be cherished. In comes Vida, with her manuscript. Her book is about her gorgeous body, in which she feels uncomfortable. The librarian makes her feel comfortable, and together they live in the back of the library until the trip to Tijuana changes them in ways neither of them had ever expected.","Fiction,Novels,Romance,Literature,Literature,American,Writing,Books About Books,Literature,20th Century,The United States Of America,Classics,Humor",3.99,396,The Abortion,5583,en,"A reclusive young man works in a San Francisco library for unpublishable books. Life's losers, an astonishing number of whom seem to be writers, can bring their manuscripts to the library, where they will be welcomed, registered and shelved. They will not be read, but they will be cherished. In comes Vida, with her manuscript. Her book is about her gorgeous body, in which she feels uncomfortable. The librarian makes her feel comfortable, and together they live in the back of the library until the trip to Tijuana changes them in ways neither of them had ever expected.","Title: The Abortion. Desc / Summary: A reclusive young man works in a San Francisco library for unpublishable books. Life's losers, an astonishing number of whom seem to be writers, can bring their manuscripts to the library, where they will be welcomed, registered and shelved. They will not be read, but they will be cherished. In comes Vida, with her manuscript. Her book is about her gorgeous body, in which she feels uncomfortable. The librarian makes her feel comfortable, and together they live in the back of the library until the trip to Tijuana changes them in ways neither of them had ever expected.. Average Rating: 3.99. Categories: Fiction,Novels,Romance,Literature,Literature,American,Writing,Books About Books,Literature,20th Century,The United States Of America,Classics,Humor. Author: Richard Brautigan."
10764,Harriet Doerr,"The long-awaited and highly praised second novel by the author of Stones for Ibarra. The American characters here find themselves waiting, hoping, and living in rural Mexico-a land with the power to enchant, repulse, captivate, and change all who pass through it. Named a Best Book of the Year by Publishers Weekly and a Notable Book of the Year by the New York Times.","Fiction,Historical,Historical Fiction,Novels,Historical,Audiobook,Literature,American,Cultural,Latin American,Book Club,Literary Fiction,Adult Fiction",3.87,116,"Consider This, SeÃ±ora",860,en,"The longawaited and highly praised second novel by the author of Stones for Ibarra. The American characters here find themselves waiting, hoping, and living in rural Mexicoa land with the power to enchant, repulse, captivate, and change all who pass through it. Named a Best Book of the Year by Publishers Weekly and a Notable Book of the Year by the New York Times.","Title: Consider This, SeÃ±ora. Desc / Summary: The longawaited and highly praised second novel by the author of Stones for Ibarra. The American characters here find themselves waiting, hoping, and living in rural Mexicoa land with the power to enchant, repulse, captivate, and change all who pass through it. Named a Best Book of the Year by Publishers Weekly and a Notable Book of the Year by the New York Times.. Average Rating: 3.87. Categories: Fiction,Historical,Historical Fiction,Novels,Historical,Audiobook,Literature,American,Cultural,Latin American,Book Club,Literary Fiction,Adult Fiction. Author: Harriet Doerr."
6672,"Carlos Fuentes,Alfred J. MacAdam","Hailed as a masterpiece since its publication in 1962, ,The Death of Artemio Cruz, is Carlos Fuentes's haunti



**Transición del Modelo 1 ('all-MiniLM-L6-v2') al Modelo 2 ('bert-large-nli-mean-tokens')**

El modelo 'all-MiniLM-L6-v2' mapea oraciones y párrafos a un espacio vectorial denso de 384 dimensiones, lo que lo hace eficiente para tareas como la agrupación o la búsqueda semántica. Sin embargo, para capturar matices semánticos más complejos, vamos a probar 'bert-large-nli-mean-tokens', el cual, según su [documentación](https://huggingface.co/sentence-transformers/bert-large-nli-mean-tokens), asigna oraciones y párrafos a un espacio vectorial denso de 1024 dimensiones. Este incremento en el número de dimensiones, y por ende en el número de parámetros, puede proporcionar una representación más rica del significado de los textos, mejorando potencialmente la calidad de las recomendaciones, aunque a costa de un mayor requerimiento computacional.



---
## 6.2. Modelo pre-entrenado **bert-large-nli-mean-tokens**

In [17]:
model_II = SentenceTransformer('sentence-transformers/bert-large-nli-mean-tokens')

book_embeddings_II = model_II.encode(df_hf['description'].tolist())

In [18]:
query = "A book to teach children about nature"
recommendations = get_recommendations(query, book_embeddings_II, df_hf,model_II, 10)
recommendations

,author,desc,genre,rating,reviews,title,totalratings,idioma,clean_description,description
1880,Karen Andreola,"This is a thorough chapter-by-chapter overview of the inspiring teaching principles of Christian educator Charlotte Mason from the original Charlotte Mason authority, speaker, and columnist, Karen Andreola. With warmth and humor, Karen provides a wealth of insight, practical advice, and narratives, illustrating how educating ""the Charlotte Mason way"" actually works. In this book, you will discover how to recognize a ""living book""; use narration; teach reading, spelling, composition, and vocabulary; appreciate great art, classical music, poetry, plays, and novels; keep a Nature Notebook for science; take part in hero-admiration for history, establish helpful habits; create a loving home atmosphere; and more.","Education,Nonfiction,Parenting,Reference,Christian,Academic,School,Family,Homeschool,Childrens,Teaching",4.26,176,Charlotte Mason Companion,2443,en,"This is a thorough chapterbychapter overview of the inspiring teaching principles of Christian educator Charlotte Mason from the original Charlotte Mason authority, speaker, and columnist, Karen Andreola. With warmth and humor, Karen provides a wealth of insight, practical advice, and narratives, illustrating how educating the Charlotte Mason way actually works. In this book, you will discover how to recognize a living book use narration teach reading, spelling, composition, and vocabulary appreciate great art, classical music, poetry, plays, and novels keep a Nature Notebook for science take part in heroadmiration for history, establish helpful habits create a loving home atmosphere and more.","Title: Charlotte Mason Companion. Desc / Summary: This is a thorough chapterbychapter overview of the inspiring teaching principles of Christian educator Charlotte Mason from the original Charlotte Mason authority, speaker, and columnist, Karen Andreola. With warmth and humor, Karen provides a wealth of insight, practical advice, and narratives, illustrating how educating the Charlotte Mason way actually works. In this book, you will discover how to recognize a living book use narration teach reading, spelling, composition, and vocabulary appreciate great art, classical music, poetry, plays, and novels keep a Nature Notebook for science take part in heroadmiration for history, establish helpful habits create a loving home atmosphere and more.. Average Rating: 4.26. Categories: Education,Nonfiction,Parenting,Reference,Christian,Academic,School,Family,Homeschool,Childrens,Teaching. Author: Karen Andreola."
16845,"Tony Johnston,Jim LaMarche","â€œA quiet, beautiful picture book to share.â€ â€”,Booklist ,(starred review), ,â€œThis gentle, lyrical celebration of the natural world will reward similarly observant readers.â€ â€”,Kirkus Reviews ,(starred review), ,â€œA touching reminder about the beauty of the natural world.â€ â€”,School Library Journal ,(starred review), ,â€œWith meditative language, Johnston offers a vivid sense of the changing seasons and of stillness. LaMarche quietly and sensitively portrays a child whoâ€™s comfortable spending hours alone, working on her own projects and observingâ€”a young naturalist.â€ â€”,Publishers Weekly ,(starred review), ,Witness the changing of a season through a watchful childâ€™s eyes in this story of nature and discovery from award-winning author Tony Johnston and ,New York Times ,Best Illustrated artist Jim La Marche.,Day after day, a girl goes to her favorite place in the woods and quietly watches from her tree house as the chipmunks, the doe, the rabbits prepare for the winter. As the temperature drops, sunset comes earlier and a new season begins. Silently she observes the world around her as it reveals its secrets. It takes time and patience to see the changes as, slowly but surely, winter comes.","Childrens,Picture Books,Animals,Environment,Nature,Childrens,Poetry,Science,Realistic Fiction,Kids,Childrens,1st Grade,Childre

**Transición del Modelo 2 ('bert-large-nli-mean-tokens') al Modelo 3 ('all-roberta-large-v1')**

A pesar de que 'bert-large-nli-mean-tokens' ya ofrece embeddings en un espacio de 1024 dimensiones, vamos a experimentar con 'all-roberta-large-v1'. Según su [documentación](https://huggingface.co/sentence-transformers/all-roberta-large-v1), este modelo también mapea oraciones y párrafos a un espacio vectorial denso de 1024 dimensiones, pero gracias a las mejoras en la arquitectura RoBERTa, pensamos que podría ofrecer una mejor capacidad para capturar contextos complejos y relaciones semánticas sutiles. Esta prueba nos permitirá comparar la eficacia de ambos modelos de 1024 dimensiones.

---
## 6.3. Modelo pre-entrenado **all-roberta-large-v1**

In [20]:
model_III = SentenceTransformer('sentence-transformers/all-roberta-large-v1')

book_embeddings_III = model_III.encode(df_hf['description'].tolist())

In [22]:
query = "A book to teach children about nature"
query_II = "A book to explore meditation and mindfulness practices"
query_III = "A book to learn mathematics from scratch"
query_IV = "A book to explore world travel adventures"
query_V = "A book to learn how to cook delicious recipes"
query_VI = "A book to understand the basics of personal finance"
query_VII = "A book to dive into the history of art"
query_VIII = "A book to master programming for beginners"
query_IX = "A book to enhance creative writing skills"
query_X = "A book to study modern psychology"
query_XI = "A book to discover the secrets of effective leadership"
query_XII = "A book to explore ancient civilizations"
recommendations = get_recommendations(query_XII, book_embeddings_III, df_hf,model_III, 10)
recommendations

,author,desc,genre,rating,reviews,title,totalratings,idioma,clean_description,description
8674,"C.W. Ceram,Sophie Wilkins,E.B. Garside","C.W. Ceram visualized archeology as a wonderful combination of high adventure, romance, history and scholarship, and this book, a chronicle of man's search for his past, reads like a dramatic narrative. We travel with Heinrich Schliemann as, defying the ridicule of the learned world, he actually unearths the remains of the ancient city of Troy. We share the excitement of Lord Carnarvon and Howard Carter as they first glimpse the riches of Tutankhamen's tomb, of George Smith when he found the ancient clay tablets that contained the records of the Biblical Flood. We rediscover the ruined splendors of the Hanging Gardens of Babylon, one of the wonders of the ancient wold; of Chichen Itza, the abandoned pyramids of the Maya: and the legendary Labyrinth of tile Minotaur in Crete. Here is much of the history of civilization and the stories of the men who rediscovered it.,From the Paperback edition.","History,Nonfiction,History,Archaeology,Science,Anthropology,History,Ancient History,Art,European Literature,German Literature,Classics,Northern Africa,Egypt",4.24,176,"Gods, Graves and Scholars: The Story of Archaeology",2081,en,"C.W. Ceram visualized archeology as a wonderful combination of high adventure, romance, history and scholarship, and this book, a chronicle of man's search for his past, reads like a dramatic narrative. We travel with Heinrich Schliemann as, defying the ridicule of the learned world, he actually unearths the remains of the ancient city of Troy. We share the excitement of Lord Carnarvon and Howard Carter as they first glimpse the riches of Tutankhamen's tomb, of George Smith when he found the ancient clay tablets that contained the records of the Biblical Flood. We rediscover the ruined splendors of the Hanging Gardens of Babylon, one of the wonders of the ancient wold of Chichen Itza, the abandoned pyramids of the Maya: and the legendary Labyrinth of tile Minotaur in Crete. Here is much of the history of civilization and the stories of the men who rediscovered it.,From the Paperback edition.","Title: Gods, Graves and Scholars: The Story of Archaeology. Desc / Summary: C.W. Ceram visualized archeology as a wonderful combination of high adventure, romance, history and scholarship, and this book, a chronicle of man's search for his past, reads like a dramatic narrative. We travel with Heinrich Schliemann as, defying the ridicule of the learned world, he actually unearths the remains of the ancient city of Troy. We share the excitement of Lord Carnarvon and Howard Carter as they first glimpse the riches of Tutankhamen's tomb, of George Smith when he found the ancient clay tablets that contained the records of the Biblical Flood. We rediscover the ruined splendors of the Hanging Gardens of Babylon, one of the wonders of the ancient wold of Chichen Itza, the abandoned pyramids of the Maya: and the legendary Labyrinth of tile Minotaur in Crete. Here is much of the history of civilization and the stories of the men who rediscovered it.,From the Paperback edition.. Average Rating: 4.24. Categories: History,Nonfiction,History,Archaeology,Science,Anthropology,History,Ancient History,Art,European Literature,German Literature,Classics,Northern Africa,Egypt. Author: C.W. Ceram,Sophie Wilkins,E.B. Garside."
7472,Simon Baker,"This is the story of the greatest empire the world has ever known. Simon Baker charts the rise and fall of the world's first superpower, focusing on six momentous turning points that shaped Roman history. Welcome to Rome as you've never seen it before - awesome and splendid, gritty and squalid.,At the heart of this gripping popular history are the dynamic, complex but flawed characters of Rome's most powerful rulers: men such as Pompey the Great, Julius Caesar, Augustus, Nero and Constantine.,The superb narrative, full of energy and imagination, is a brilliant distillati

In [23]:
import pickle
from sentence_transformers import SentenceTransformer


# Modelo 1: all-MiniLM-L6-v2
model_allMiniLM = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings_allMiniLM = model_allMiniLM.encode(df_hf['description'].tolist())

with open('modelos/model_allMiniLM.pkl', 'wb') as f:
    pickle.dump(model_allMiniLM, f)
with open('modelos/embeddings_allMiniLM.pkl', 'wb') as f:
    pickle.dump(embeddings_allMiniLM, f)


# Modelo 2: bert-large-nli-mean-tokens
model_bertLarge = SentenceTransformer('sentence-transformers/bert-large-nli-mean-tokens')
embeddings_bertLarge = model_bertLarge.encode(df_hf['description'].tolist())

with open('modelos/model_bertLarge.pkl', 'wb') as f:
    pickle.dump(model_bertLarge, f)
with open('modelos/embeddings_bertLarge.pkl', 'wb') as f:
    pickle.dump(embeddings_bertLarge, f)


# Modelo 3: all-roberta-large-v1
model_robertaLarge = SentenceTransformer('sentence-transformers/all-roberta-large-v1')
embeddings_robertaLarge = model_robertaLarge.encode(df_hf['description'].tolist())

with open('modelos/model_robertaLarge.pkl', 'wb') as f:
    pickle.dump(model_robertaLarge, f)
with open('modelos/embeddings_robertaLarge.pkl', 'wb') as f:
    pickle.dump(embeddings_robertaLarge, f)

---
## 6.4. Estudio del Error

En el campo de los sistemas de recomendación, la evaluación del error es un desafío particular, ya que no se cuenta con métricas directas y universales como en problemas de clasificación. Durante el desarrollo del proyecto, hemos estudiado diversas técnicas de evaluación para recomendadores, y a continuación, presentamos un resumen de estas metodologías.

### Evaluación Offline

En un primer enfoque, se pueden utilizar evaluaciones offline, que consisten en dividir el dataset en conjuntos de entrenamiento y prueba. Aunque esta metodología es común en tareas de clasificación, en sistemas de recomendación se adaptan métricas específicas, tales como:

- **Hit Rate y Precision@K:** Miden la frecuencia con la que los ítems recomendados incluyen aquellos que el usuario ha consumido o valorado positivamente.
- **NDCG (Normalized Discounted Cumulative Gain):** Valora la posición de los ítems relevantes en la lista de recomendaciones, penalizando aquellos que aparecen en posiciones inferiores.
- **MAP (Mean Average Precision):** Proporciona una medida global de la precisión, considerando la relevancia de las recomendaciones en distintos niveles.

Estas métricas ofrecen una valoración cuantitativa de la calidad del sistema sin requerir pruebas en un entorno en vivo.

### A/B Testing

En este método, se presentan diferentes versiones del recomendador a distintos grupos de usuarios y se comparan sus rendimientos basados en métricas reales, como:

- **Tasa de Conversión:** Proporción de usuarios que realizan una acción deseada, por ejemplo, seleccionar un libro recomendado.
- **Tiempo de Interacción:** Duración de la sesión o tiempo dedicado a explorar las recomendaciones.
- **Feedback Directo:** Valoraciones o comentarios proporcionados por los usuarios sobre la utilidad de las recomendaciones.

El A/B testing permite validar en condiciones reales cómo interactúan los usuarios con el sistema y ajustar, en consecuencia, parámetros o estrategias de recomendación.

### Feedback en Tiempo Real y Modelos de Reinforcement Learning

Otra línea de investigación que hemos estudiado es el uso del feedback en tiempo real, el cual busca adaptar las recomendaciones en función de la interacción inmediata del usuario. Este enfoque puede incluir técnicas de **reinforcement learning**, donde el sistema aprende a optimizar sus sugerencias basándose en recompensas derivadas de comportamientos como clicks o conversiones. Esta técnica muestra un potencial elevado para mejorar la personalización y actualmente se encuentra en una fase exploratoria para muchos escenarios de recomendación.

### Consideraciones Finales

Si bien hemos investigado estas técnicas de evaluación para sistemas de recomendación, un análisis exhaustivo del error en este contexto resulta complejo y se escapa del alcance de este trabajo. Por ello, hemos decidido enfocar nuestro estudio del error en el clasificador basado en LSTM, donde contamos con métricas claras como precisión y pérdida.

En resumen, hemos estudiado metodologías como la evaluación offline, A/B testing y el feedback en tiempo real para mejorar la calidad de los recomendadores. Sin embargo, dado que estas técnicas requieren un enfoque más profundo y específico, las consideramos líneas de investigación futuras para perfeccionar nuestros sistemas de recomendación.